**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

**To do:**
- Add dropout
- Implement another loss function

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
name = 'bpi_2012'
#name = 'bpi_2013'

parser = {
    'train': True,
    'test': True,
    'model_class': 'VAE',
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name), 
    'batch_size' : 16,
    'epochs' : 50,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 1000,
    'layer2': 100,
    'early_stopping': 30,
    'lr': 0.0005,
    'betas': (0.9, 0.99),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}_{2}/'.format(name, args.nan_pct, args.model_class)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

## Validate and test

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [14]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [15]:
missing_true_val.shape, missing_true_test.shape

((55671, 5), (47823, 5))

In [16]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [17]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [18]:
pd.isnull(normalized_missing_df).sum()

CaseID                   0
Activity             78944
CompleteTimestamp    78376
CumTimeInterval      78376
NormalizedTime       78376
dtype: int64

In [19]:
pd.isnull(missing_true_val).sum()

CaseID                   0
Activity             16971
CompleteTimestamp    16546
CumTimeInterval      16546
NormalizedTime       16546
dtype: int64

In [20]:
pd.isnull(missing_true_test).sum()

CaseID                   0
Activity             14349
CompleteTimestamp    14501
CumTimeInterval      14501
NormalizedTime       14501
dtype: int64

# Build model

## Define model

In [21]:
if args.model_class == 'VAE':
    model = VAE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2, args.cuda)
if args.model_class == 'VAE_dropout':
    model = VAE_dropout(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2, args.cuda)
    
if args.cuda:
    model.cuda()

## Define loss

In [22]:
# Define loss

def loss_function(recon_x, x, mu, logvar, avai_mask): 
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False)
    #MSE = F.mse_loss(recon_x*avai_mask, x*avai_mask, size_average=False)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    #loss = MSE+KLD
    loss = BCE+KLD
    return loss

In [23]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [24]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
lambda1 = lambda epoch: epoch//10
#lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [25]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [26]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [27]:
def val(model, missing_matrix_w_normalized_time_val, 
        complete_matrix_w_normalized_time_val, 
        avai_matrix_val, nan_matrix_val):
    
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    avai_matrix_val = Variable(torch.Tensor(avai_matrix_val).float())
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        avai_matrix_val = avai_matrix_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data, mu, logvar = model(m_val)
        
    recon = getProbability(recon_data)
    
    if args.cuda:
        recon_2d = recon.cuda().view(recon.size(0)*recon.size(1), recon.size(2))
    else:
        recon_2d = recon.view(recon.size(0)*recon.size(1), recon.size(2))
    gt_2d = c_val.view(recon.size(0)*recon.size(1), recon.size(2))
    avai_matrix_2d = avai_matrix_val.view(recon.size(0)*recon.size(1), recon.size(2))
    nan_matrix_2d = nan_matrix_val.view(recon.size(0)*recon.size(1), recon.size(2))
    
    submission = torch.add(recon_2d*nan_matrix_2d, gt_2d*avai_matrix_2d)
    
    #Time
    predicted_time = submission.data[:, 0].cpu().numpy()
    gt_time = gt_2d.data[:, 0].cpu().numpy()
    mae = mean_absolute_error(gt_time, predicted_time)
    
    #Activity
    predicted_act = submission.data[:, 1:].cpu().numpy()
    gt_act = gt_2d.data[:, 1:].cpu().numpy()
    
    a = predicted_act[np.any(predicted_act != 0, axis=1)]
    b = gt_act[np.any(gt_act != 0, axis=1)]   
    
    predicted_labels = np.argmax(a, axis=1)
    gt_labels = np.argmax(b, axis=1)
    acc = accuracy_score(gt_labels, predicted_labels)
    
    return mae+1/acc

In [28]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.524752e-01
2,7853,NaN,NaN,NaN,NaN
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:59:15.103,8292030.557,5.115728e-07


In [29]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.624972e-03
2,7853,A_DECLINED-COMPLETE,2012-01-05 06:57:55.323,8291950.777,9.974918e-01
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:59:15.103,8292030.557,5.115717e-07


# Train model

In [30]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader,
                                                                complete_matrix_w_normalized_time_trainLoader,
                                                                avai_matrix_trainLoader)):
        
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())
      
        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()
            
        optimizer.zero_grad()
        
        recon_data, mu, logvar = model(m_data)
        
        loss = loss_function(recon_data, c_data, mu, logvar, avai_mask)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [31]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        scheduler.step()
        
        train_loss = train(epoch, model, optimizer)
        end_train = time.time()
        
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, 
                        avai_matrix_val, nan_matrix_val)
        save_model(model, epoch, val_score)
        
        '''
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        '''
        #method 2 scheduler
        #scheduler.step(val_score)
        
        end = time.time()
        print('====> Epoch {} | Train time: {:.4f} ms| End time: {:.4f} ms | Train loss: {:.4f} | Val score: {:.4f}'.
              format(epoch, (end_train-init)*1000, (end-init)*1000, train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Train time: 3314.4324 ms| End time: 7171.6073 ms | Train loss: 371.5301 | Val score: 1.4344
====> Epoch 2 | Train time: 3230.6919 ms| End time: 6821.5230 ms | Train loss: 371.4983 | Val score: 1.4371
====> Epoch 3 | Train time: 3220.7222 ms| End time: 6797.8146 ms | Train loss: 371.4634 | Val score: 1.4362
====> Epoch 4 | Train time: 3222.5547 ms| End time: 6831.9991 ms | Train loss: 371.5192 | Val score: 1.4369
====> Epoch 5 | Train time: 3217.5324 ms| End time: 6788.3267 ms | Train loss: 371.4793 | Val score: 1.4367
====> Epoch 6 | Train time: 3220.2439 ms| End time: 6802.6752 ms | Train loss: 371.5772 | Val score: 1.4376
====> Epoch 7 | Train time: 3223.4502 ms| End time: 6783.9818 ms | Train loss: 371.5382 | Val score: 1.4383
====> Epoch 8 | Train time: 3227.0749 ms| End time: 6804.9827 ms | Train loss: 371.5421 | Val score: 1.4366
====> Epoch 9 | Train time: 3230.2649 ms| End time: 6805.5336 ms | Train loss: 371.5139 | Val score: 1.4359
====> Epoch 10 | Train time:

# Predict and evaluate

In [32]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test, mu, logvar = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    
    print('Fixing Time...')
    submission = fixTime(submission_df)
        
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    
    print('\n')
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...
Fixing Time...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 14501
Mean Absolute Error: 1.0616 day(s)
Root Mean Squared Error: 2.2944 day(s)
Number of missing Activity: 14349
Accuracy: 68.22%


Saving submission...
Done!


In [33]:
submission_df.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.111216
2,10470,A_DECLINED-COMPLETE,A_DECLINED-COMPLETE,1.086354e+07,1.086354e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:42.964
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_PREACCEPTED-COMPLETE,1.086394e+07,1.086394e+07,2012-02-04 01:24:23.379,2012-02-04 01:24:23.379
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086394e+07,2012-02-04 01:24:24.052,2012-02-04 01:24:24.052
7,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087644e+07,1.087644e+07,2012-02-04 04:52:43.090,2012-02-04 04:52:43.090
8,10471,W_Completeren aanvraag-COMPLETE,W_Completeren aanvraag-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.087697e+07,2012-02-04 05:01:31.267,2012-02-04 05:01:31.267


In [34]:
submission.head(20)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.111216
2,10470,A_DECLINED-COMPLETE,A_DECLINED-COMPLETE,1.086354e+07,1.086354e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:42.964
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_PREACCEPTED-COMPLETE,1.086394e+07,1.086394e+07,2012-02-04 01:24:23.379,2012-02-04 01:24:23.379
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086394e+07,2012-02-04 01:24:24.052,2012-02-04 01:24:24.052
7,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087644e+07,1.087644e+07,2012-02-04 04:52:43.090,2012-02-04 04:52:43.090
8,10471,W_Completeren aanvraag-COMPLETE,W_Completeren aanvraag-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.087697e+07,2012-02-04 05:01:31.267,2012-02-04 05:01:31.267


In [35]:
missing_true_test.head(20)

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,10470,A_SUBMITTED-COMPLETE,2012-02-04 01:17:11.047,1.086351e+07,0.000000e+00
1,10470,NaN,NaN,NaN,NaN
2,10470,A_DECLINED-COMPLETE,2012-02-04 01:17:42.964,1.086354e+07,9.968767e-01
3,10471,A_SUBMITTED-COMPLETE,2012-02-04 01:23:41.949,1.086390e+07,0.000000e+00
4,10471,A_PARTLYSUBMITTED-COMPLETE,2012-02-04 01:23:42.504,1.086390e+07,4.152251e-05
5,10471,A_PREACCEPTED-COMPLETE,2012-02-04 01:24:23.379,1.086394e+07,3.099599e-03
6,10471,W_Completeren aanvraag-SCHEDULE,2012-02-04 01:24:24.052,1.086394e+07,3.149950e-03
7,10471,W_Completeren aanvraag-START,2012-02-04 04:52:43.090,1.087644e+07,9.382696e-01
8,10471,W_Completeren aanvraag-COMPLETE,2012-02-04 05:01:19.935,1.087696e+07,9.769375e-01
9,10471,W_Completeren aanvraag-START,2012-02-04 05:01:31.267,1.087697e+07,9.777853e-01


In [36]:
first_timestamp

Timestamp('2011-10-01 07:38:44.546000')